<a href="https://colab.research.google.com/github/alexrods/cars-bikes-detection/blob/main/Notebooks/2_0_json_tfrecord.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Extract Json data

In [ ]:
import json 
import pickle
import zipfile
import pandas as pd
import urllib.parse as up

!pip install tf_slim
# !pip install tensorflow-object-detection-api

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
local_zip = "/content/dataset_original_2.zip"
zip_ref = zipfile.ZipFile(local_zip, "r")
zip_ref.extractall("dataset_original")
zip_ref.close()

In [ ]:
type_file = "test"
path = "/content/test_dataset2.0.json"
data_file = open(path)
data = json.load(data_file)

In [ ]:
import regex as rx

In [ ]:
csv_list = []
'''
Recorremos la estructura del archivo json, extrayendo las variables relevantes
1. Clase
2. Bounding box (x inicial, y inicial, x final, y final)
3. Ancho y alto de la imagen.
4. Nombre del archivo.
'''
for classification in data:
  width, height = classification['width'], classification['height']
  image = up.unquote(classification['image']).replace("+", " ")
  for item in classification['tags']:
    name = item['name']
    xmin = item['pos']['x']
    ymin = item['pos']['y']
    xmax = item['pos']['x'] + item['pos']['w']
    ymax = item['pos']['y'] + item['pos']['h']

    value = (image, width, height, name, xmin, ymin, xmax, ymax)
    csv_list.append(value)

# Al final lo almacenamos en un CSV con el nombre de las columnas siguiente
column_name = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax']
csv_df = pd.DataFrame(csv_list, columns = column_name)

# Convertimos el dataframe a CSV
csv_df.to_csv("/content/{}_labels.csv".format(type_file))

In [ ]:
csv_df

,filename,width,height,class,xmin,ymin,xmax,ymax
0,3 (28).png,960,540,carro,391.500000,246.500000,541.500000,353.500000
1,3 (28).png,960,540,carro,547.500000,193.500000,679.500000,296.500000
2,3 (28).png,960,540,carro,375.500000,131.500000,455.500000,204.500000
3,3 (28).png,960,540,carro,244.500000,143.500000,308.500000,181.500000
4,4 (4).png,1280,720,carro,262.134672,500.280425,393.476914,624.325876
...,...,...,...,...,...,...,...,...
446,3 (8).png,960,540,carro,122.500000,146.500000,183.500000,186.500000
447,3 (6).png,960,540,carro,317.500000,130.500000,388.500000,188.500000
448,3 (6).png,960,540,carro,255.500000,149.500000,355.500000,219.500000
449,3 (6).png,960,540,carro,393.500000,368.500000,578.500000,527.500000


In [ ]:
csv_df = pd.DataFrame(csv_list, columns=column_name)
csv_df.to_csv("/content/{}_labels.csv".format(type_file))

## Transform CSV to TFRecord

In [ ]:
import os
%cd /content
!git clone --quiet https://github.com/tensorflow/models.git
%cd /content/models/
!git checkout 58d19c67e1d30d905dd5c6e5092348658fed80af
!apt-get update && apt-get install -y -qq protobuf-compiler python-pil python-lxml python-tk
!pip install -q Cython contextlib2 pillow lxml matplotlib
!pip install -q pycocotools
%cd /content/models/research
!protoc object_detection/protos/*.proto --python_out=.
os.environ['PYTHONPATH'] += ':/content/models/research/:/content/models/research/slim/'
!python object_detection/builders/model_builder_test.py

/content
fatal: destination path 'models' already exists and is not an empty directory.
/content/models
HEAD is now at 58d19c67e Internal change
Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  Release
Get:5 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:6 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Get:9 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:12 http://archive.ubuntu.com/ubuntu focal-backports In

In [ ]:
from __future__ import division
from __future__ import print_function
from __future__ import absolute_import

import os
import io
import pandas as pd
import tensorflow as tf
import sys
sys.path.append("../../models/research")

from PIL import Image
from object_detection.utils import dataset_util
from collections import namedtuple, OrderedDict


# Es fundamental remplazar las clases con las mismas clases del proyecto.
# Los nombres deben estar escritos exactamente igual.
# Si existen más clases en tu proyecto agregarias un ELIF.
def class_text_to_int(row_label):
    if row_label == "carro": 
      return 1
    elif row_label == "motos":
      return 2
    else:
        None


def split(df, group):
    data = namedtuple('data', ['filename', 'object'])
    gb = df.groupby(group)
    return [data(filename, gb.get_group(x)) for filename, x in zip(gb.groups.keys(), gb.groups)]


def create_tf_example(group, path):
    with tf.io.gfile.GFile(os.path.join(path, '{}'.format(group.filename)), 'rb') as fid:
        encoded_jpg = fid.read()
    encoded_jpg_io = io.BytesIO(encoded_jpg)
    image = Image.open(encoded_jpg_io)
    width, height = image.size

    filename = group.filename.encode('utf8')
    image_format = b'jpg'
    # check if the image format is matching with your images.
    xmins = []
    xmaxs = []
    ymins = []
    ymaxs = []
    classes_text = []
    classes = []

    for index, row in group.object.iterrows():
        xmins.append(row['xmin'] / width)
        xmaxs.append(row['xmax'] / width)
        ymins.append(row['ymin'] / height)
        ymaxs.append(row['ymax'] / height)
        classes_text.append(row['class'].encode('utf8'))
        classes.append(class_text_to_int(row['class']))

    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(filename),
        'image/source_id': dataset_util.bytes_feature(filename),
        'image/encoded': dataset_util.bytes_feature(encoded_jpg),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    return tf_example

# Modificamos con la ubicación de nuestros archivos. Recuerda tomarte el tiempo 
# de revisar que la URL sea la misma que la de Google Colab en donde subiste el 
# archivo de forma manual
output_path = "train.record"
image_dir = "/content/dataset_original/dataset_original_2"
csv_input = "/content/train_labels.csv"

writer = tf.io.TFRecordWriter(output_path)
path = os.path.join(image_dir)
examples = pd.read_csv(csv_input)
grouped = split(examples, 'filename')
for group in grouped:
    tf_example = create_tf_example(group, path)
    writer.write(tf_example.SerializeToString())

writer.close()
output_path = os.path.join(os.getcwd(), output_path)
print('Successfully created the TFRecords: {}'.format(output_path))


Successfully created the TFRecords: /content/models/research/train.record
